# Разработка базы данных на примере софтверной компании BrunoyamSoft
Компания **BrunoyamSoft** занимается разработкой мобильных приложений с сфере аналитики данных, развлечений и бизнес-приложений. Компания широко представлена в маркетплейсах основных мобильных платформ и имеет большой портфель приложений различной тематики для удовлетворения требовательных пользователей.

Для контроля эффективности работы компании, была создана специальная база данных. База должна решать следующие задачи:
- Осуществлять контроль динамики продаж мобильных приложений и предоставлять аналитику по продажам. Хранить необходимые данные для этого;
- Осуществлять контроль над разработкой приложений. Связывать персонал с приложениями и их продажами.
- Предоставлять гибкие механизмы для конструирования произвольных запросов SQL в целях дальнейшего развития аналитических шаблонов.

### Разработаем схему хранения данных компании BrunoyamSoft.

База данных содержит следующие основные таблицы описывающие бизнес-логику компании
- **programmers** - таблица содержит список программистов, каждый из которых имеет имя, фамилию, а также ему присвоена квалификация;
- **languages** - таблица содержит список языков программирования, на которых разрабатываются приложения в компании;
- **qualification** - таблица содержит возможные квалификации, присваемые каждому программисту;
- **products** - таблица содержит разработанные программные продукты, их имя а также идентификатор OS для которой продукт разработан;
- **category** - содержит возможные значения категорий которые могут быть присвоены продукту;
- **os** - содержит возможные значения идентификаторов операционных систем (платформ). Каждый продукт может быть только для одной OS;
- **purshases** - таблица записей продаж. Содержит дату продажи и идентификатор продукта.
    
По мимо основных таблиц, существуют дополнительные связывающие таблицы, определяющие следующие требования:
- Каждый программист может знать несколько языков программирования. Один язык программирования может знать несколько программистов;
- Каждый программист может работать над несколькими продуктами. Один продукт как правило разрабатывает несколько программистов.

Для реализации данных требований, используется связь _```многие-ко-многим```_, для чего в базу данных добавлены следующие связывающие таблицы:
- **lang_knowledge** - таблица связывает программистов и языки программирования, которые они знают;
- **dev_products** - таблица связывает программистов и продукты которые они разрабатывают.

**Представим разработанную схему графически:**



![aa](./schema.png)

**Загрузим поддержку SQLite в Python и создадим указанные таблицы с помощью Pandas**

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path
import random
from datetime import datetime, date, time

Создадим функцию загрузки базы данных. Если база создается впервые, то выполняется скрипт ```init.sql``` по созданию таблиц и добавлению базовых записей в ней.

In [2]:
def get_sql():
    sql = sqlite3.connect('brunoyam.db')
    query = pd.read_sql('SELECT * FROM sqlite_master', sql)
    if query.name.count() > 0:
        return sql
    script = Path('init.sql').read_text()    
    sql.cursor().executescript(script)
    return sql

Создадим функцию ```init_purchares``` добавления записей продаж. Эти записи будут анализироваться в дальнейшем. Записи генерируются синтетические и только один раз - после создания таблицы. 

In [3]:
# Количество продаж в базе, которые требуется сгенерировать при первом запуске
COUNT_PURSHARES = 1024

In [4]:
def gen_newpurshase(price_table):
    year = random.randint(2016, 2021)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    hour = random.randint(1, 23)
    m = random.randint(1, 59)
    d = datetime(year, month, day, hour, m)
    prod_id = random.randint(1, price_table['id'].count())
    price =  price_table['curr_price'][prod_id-1]
    return {'date' : d, 'product_id' : prod_id, 'price' : price}

def get_newpurchases(count, sql):
    products = pd.read_sql('SELECT * FROM instance_products', sql)
    records = []   
    for x in range(count):
        record = gen_newpurshase(products)
        records.append(record)
    frame = pd.DataFrame(records)
    return frame

def init_purchares(sql):
    purchases = pd.read_sql('SELECT * FROM purchases', sql)
    if purchases.id.count() > 0:
        return
    records = get_newpurchases(COUNT_PURSHARES, sql)
    nrecords = pd.DataFrame(records)
    nrecords.to_sql('purchases', sql, index=False, if_exists='append')
    sql.commit()
sql = get_sql()
init_purchares(sql)

Получим список продаж отсортированный по дате

In [5]:
purchases = pd.read_sql(
    """
    SELECT date, products.name, price FROM 'purchases' 
    JOIN 'instance_products' ON instance_products.id = product_id 
    JOIN 'products' ON products.id = instance_products.prod_id 
    ORDER BY date
    """,
     sql)
purchases

,date,name,price
0,2016-01-01 10:46:00,Funny Office,34.9
1,2016-01-02 03:46:00,Clev Calc,0.0
2,2016-01-03 02:23:00,Funny Office,34.9
3,2016-01-03 12:56:00,Mimo,10.0
4,2016-01-03 23:21:00,Funny Office,9.9
...,...,...,...
1019,2021-12-20 10:23:00,Mimo,10.0
1020,2021-12-22 14:15:00,Clev Calc,0.0
1021,2021-12-22 15:39:00,Funny Office,34.9
1022,2021-12-23 07:49:00,Funny Office,9.9


Получим список программистов, сгрупированных по продуктам, которые они разрабатывают 

In [8]:
programmers_by_product = pd.read_sql("""
SELECT products.name as 'Product Name', os.name as OS, programmers.pname
FROM 'dev_products' 
JOIN 'instance_products' ON instance_products.id = product_id
JOIN 'products' ON products.id = instance_products.prod_id
JOIN 'os' ON os.id = instance_products.os_id
JOIN 'programmers' ON programmers.id = programmers_id
""", sql)
programmers_by_product

,Product Name,OS,pname
0,Crossout Mobile,Android,Steve Wozniak
1,Tik Fok,iPhone,Guido van Rossum
2,Tik Fok,iPhone,Steve Wozniak
3,Tik Fok,iPhone,Bill Gates
4,Mimo,Android,Steve Jobs
5,Mimo,Android,Bill Gates
6,Mimo,iPhone,Steve Jobs
7,Mimo,iPhone,Bill Gates
8,EdX,SymphonyOS,Bjarne Stroustrup
9,Sim Soti,Android,Bjarne Stroustrup


Посчитаем, какие программисты компании сколько языков программирования знают:

In [7]:
query = pd.read_sql("""
SELECT programmers.pname as 'Programmer', COUNT(language_id) as 'Lang. Knowing'
FROM 'lang_knowledge' 
JOIN 'languages' ON languages.id = language_id
JOIN 'programmers' ON programmers.id = programmers_id
GROUP BY Programmer
ORDER BY pname
""", sql)
query

,Programmer,Lang. Knowing
0,Bill Gates,1
1,Bjarne Stroustrup,1
2,Guido van Rossum,2
3,Steve Jobs,1
4,Steve Wozniak,2


Создадим обратный запрос: какой язык сколько программистов в компании знает

In [9]:
query = pd.read_sql("""
SELECT languages.name as 'Lang. Name', COUNT(programmers.pname) as 'Programmers'
FROM 'lang_knowledge' 
JOIN 'programmers' ON programmers.id = programmers_id
JOIN 'languages' ON languages.id = language_id
GROUP BY languages.name
""", sql)
query

,Lang. Name,Programmers
0,C#,2
1,C++,1
2,Java,2
3,Python,2


Получим список продуктов, над которыми работает хотя бы один сеньор

In [25]:
programmers_by_product = pd.read_sql("""
SELECT products.name as Product, programmers.pname as 'Programmer'
FROM 'dev_products' 
JOIN 'products' ON products.id = product_id
JOIN 'programmers' ON programmers.id = programmers_id
WHERE programmers.qualification_id = 3
ORDER BY pname
""", sql)
programmers_by_product

,Product,Programmer
0,Sim Soti,Bjarne Stroustrup
1,Pazzle,Bjarne Stroustrup
2,Tik Fok,Guido van Rossum
3,Funny Office,Guido van Rossum


Построим аналитику продаж. Расчитаем сумму продаж по каждому продукту на всех платформах. Бесплатные продукты с нулевым уровнем продаж исключим из аналитики

In [26]:
purchases = pd.read_sql("""
SELECT name as 'Product', SUM(price) as 'Total, $'
FROM 'purchases' 
JOIN 'products' ON products.id = product_id 
WHERE price > 0
GROUP BY name
ORDER BY price DESC
""", sql)
purchases 

,Product,"Total, $"
0,EdX,2130.0
1,Tik Fok,1168.2
2,Mimo,1010.0
3,Clev Calc,881.1
4,Crossout Mobile,533.6
5,Sim Soti,95.0


Сделаем аналогичный расчет, но по мобильным платформам. Рассчитаем сумму продаж для каждой мобильной платформы в отдельности

In [27]:
purchases = pd.read_sql("""
SELECT os.name as Platform, SUM(price) as 'Total, $'
FROM 'purchases' 
JOIN 'instance_products' ON instance_products.id = product_id 
JOIN 'os' ON os.id = instance_products.os_id 
WHERE price > 0
GROUP BY os.name
ORDER BY price DESC
""", sql)
purchases 

,Platform,"Total, $"
0,iPhone,6648.6
1,Android,2424.7
2,SymphonyOS,95.0


Расчитаем количество продаж по категориям продуктов. Также добавим дополнительную колонку ```Qty```, показывающую сколько единиц товара было продано в каждой категории

In [28]:
purchases = pd.read_sql("""
SELECT category.name as Category, SUM(price) as 'Total, $', COUNT(category.name) as Qty
FROM 'purchases' 
JOIN 'products' ON products.id = product_id 
JOIN 'category' ON category.id = products.category_id 
WHERE price > 0
GROUP BY category.name
ORDER BY price DESC
""", sql)
purchases 

,Category,"Total, $",Qty
0,Office,4021.1,290
1,Game,1796.8,277
